<a href="https://colab.research.google.com/github/jchen0000/widsdatathon2025/blob/main/python/Jiaqi_GIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns

# Data Input

In [5]:
# prompt: Access data from google drive
from google.colab import drive
drive.mount('/content/drive')

combined_metadata_imputed = pd.read_csv('/content/drive/My Drive/widsdatathon2025/Work/Processed Data/new_imputed_train.csv')
combined_test_metadata_imputed = pd.read_csv('/content/drive/My Drive/widsdatathon2025/Work/Processed Data/imputed_test_jiaqi.csv')

# Import solution w outcome data (1213, 3)
train_sol = pd.read_excel("drive/My Drive/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

# Import train MRI data (1213, 19901)
train_mri_filepath = "drive/My Drive/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv"
train_mri = pd.read_csv(train_mri_filepath)

# Remove first line, convert csv file to a functional connectome matrix
train_matrix_data = train_mri.iloc[:, 1:].to_numpy(dtype=float)
# matrix_data.shape (1213, 19900)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Graph Isomorphism Network (GIN)



In [7]:
!pip install spektral

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 2.5 MB/s eta 0:00:00


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from spektral.layers import GCNConv
from spektral.data import Graph
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from spektral.layers import GINConv
from scipy.spatial.distance import squareform
import scipy.sparse as sp

def connectivity_vector_to_graph(vector, n=200):
    """
    Converts a connectivity vector (flattened upper-triangular part of a 200x200 matrix)
    into a graph representation:
      - X: Node feature matrix (all ones, shape: [n, 1])
      - A: Adjacency matrix (dense, shape: [n, n])
    """
    A = squareform(vector)
    np.fill_diagonal(A, 0)
    X = np.ones((n, 1))
    A = A.astype(np.float32)
    return X, A

def read(connectivity_data):
        outputX = []
        outputA = []
        n_samples = connectivity_data.shape[0]
        for i in range(n_samples):
            X, A_dense = connectivity_vector_to_graph(connectivity_data[i])
            A_sparse = sp.csr_matrix(A_dense)
            outputX.append(X)
            outputA.append(A_sparse)

        outputX = np.stack(outputX)

        return outputX, outputA

connectivity_data = train_matrix_data
y_train = train_sol[['ADHD_Outcome', 'Sex_F']].values

read(connectivity_data)
X, A = read(connectivity_data)
print(X.shape)    # (1213, 200, 1)
print(y_train.shape)    # (1213, 2)

(1213, 200, 1)
(1213, 2)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from spektral.layers import GCNConv
from spektral.data import Graph
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from spektral.layers import GINConv
from scipy.spatial.distance import squareform
import scipy.sparse as sp

def connectivity_vector_to_graph(vector, n=200):
    """
    Converts a connectivity vector (flattened upper-triangular part of a 200x200 matrix)
    into a graph representation:
      - X: Node feature matrix (all ones, shape: [n, 1])
      - A: Adjacency matrix (dense, shape: [n, n])
    """
    A = squareform(vector)
    np.fill_diagonal(A, 0)
    X = np.ones((n, 1))
    A = A.astype(np.float32)
    return X, A

def read(connectivity_data):
        outputX = []
        outputA = []
        n_samples = connectivity_data.shape[0]
        for i in range(n_samples):
            X, A_dense = connectivity_vector_to_graph(connectivity_data[i])
            A_sparse = sp.csr_matrix(A_dense)
            outputX.append(X)
            outputA.append(A_sparse)

        # outputX = np.stack(outputX)

        return outputX, outputA

class GIN(Model):

  def __init__(self):
    super().__init__()
    self.Gconv1 = GINConv(32, activation='relu', mlp_hidden=[32])
    self.Gconv2 = GINConv(32, activation='relu', mlp_hidden=[32])
    self.Gconv3 = GINConv(32, activation='relu', mlp_hidden=[32])
    self.normalization = BatchNormalization()
    self.dropout = Dropout(0.65)
    self.pool = keras.layers.GlobalAveragePooling1D()
    self.dense1 = Dense(64, activation='relu')
    self.dense2 = Dense(32, activation='relu')
    self.dense3 = Dense(2, activation='sigmoid')

  def call(self, inputs):
    x, adj = inputs
    # x, adj = graph.x, graph.a
    out = self.Gconv1([x, adj])
    out = self.dropout(out)
    out = self.normalization(out)
    out = self.Gconv2([out, adj])
    out = self.dropout(out)
    out = self.normalization(out)
    out = self.Gconv3([out, adj])
    # out = self.dropout(out)
    # out = self.normalization(out)
    out = self.pool(out)
    # out = self.dense1(out)

    # out = self.dropout(out)
    out = concatenate([out, meta])

    out = self.dense1(out)
    out = self.dropout(out)
    out = self.dense3(out)

    return out

model = GIN()
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1) #, weight_decay=0.0005)
f1_scorer = tf.keras.metrics.F1Score(average='weighted')
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', f1_scorer])

model.summary()

# Train the model
connectivity_data = train_matrix_data
y_train = train_sol[['ADHD_Outcome', 'Sex_F']].values

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
X, A = read(connectivity_data)

model.fit([X, A], y_train,
          epochs=100, batch_size=32, validation_split=0.2,
          # sample_weight=weights,
          callbacks=[reduce_lr]) #reduce_lr

Model: "gin"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gin_conv (GINConv)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gin_conv_1 (GINConv)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gin_conv_2 (GINConv)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:1387: UserWarning: Layer 'gin' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Exception encountered when calling GINConv.call().

'tuple' object has no attribute 'shape'

Arguments received by GINConv.call():
  • inputs=[('tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 't

AttributeError: Exception encountered when calling GINConv.call().

[1m'tuple' object has no attribute 'shape'[0m

Arguments received by GINConv.call():
  • inputs=[('tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shape=(32, 1), dtype=float32)', 'tf.Tensor(shap

Keras

In [42]:
import numpy as np
import tensorflow as tf
from scipy.spatial.distance import squareform
import scipy.sparse as sp
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Reshape, Activation, Lambda
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from spektral.data import Graph, Dataset, DisjointLoader
from spektral.layers import GINConv, GlobalSumPool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# ---------------------------
# 1. Helper Function: Convert a connectivity vector to a graph
# ---------------------------
def connectivity_vector_to_graph(vector, n=200):
    """
    Converts a connectivity vector (flattened upper-triangular part of a 200x200 matrix)
    into a graph representation:
      - X: Node feature matrix (all ones, shape: [n, 1])
      - A: Adjacency matrix (dense, shape: [n, n])
    """
    A = squareform(vector)
    np.fill_diagonal(A, 0)
    X = np.ones((n, 1))
    A = A.astype(np.float32)
    return X, A


# ---------------------------
# 2. Custom Spektral Dataset
# ---------------------------
class FMRI_Dataset(Dataset):
    def __init__(self, connectivity_data, targets, **kwargs):
        """
        connectivity_data: NumPy array of shape (n_samples, 19900)
        targets: NumPy array of shape (n_samples, 2) with columns [ADHD_Outcome, Sex_F]
        """
        self.connectivity_data = connectivity_data
        self.targets = targets
        super().__init__(**kwargs)

    def read(self):
        output = []
        n_samples = self.connectivity_data.shape[0]
        for i in range(n_samples):
            X, A_dense = connectivity_vector_to_graph(self.connectivity_data[i])
            # Convert dense adjacency to a SciPy CSR sparse matrix (Spektral accepts this)
            A_sparse = sp.csr_matrix(A_dense)
            y = self.targets[i]
            output.append(Graph(x=X, a=A_sparse, y=y))
        return output

# ---------------------------
# 3. Prepare Training Data
# ---------------------------
# Assume train_mri has shape (1213, 19901): first column is participant ID, next 19900 columns are connectivity.
train_matrix_data = train_mri.iloc[:, 1:].to_numpy(dtype=float)  # shape: (1213, 19900)

# Extract targets from train_sol:
y_adhd = train_sol['ADHD_Outcome'].values  # shape: (1213,)
y_sex  = train_sol['Sex_F'].values         # shape: (1213,)
y_target = np.column_stack((y_adhd, y_sex))  # shape: (1213, 2)

# Create the dataset:
dataset = FMRI_Dataset(train_matrix_data, y_target)

# ---------------------------
# 4. Split Dataset into Training and Validation Sets
# ---------------------------
n_samples = len(dataset)
train_idx, val_idx = train_test_split(np.arange(n_samples), test_size=0.2, random_state=42)
dataset_train = dataset[train_idx]
dataset_val = dataset[val_idx]

# Create DisjointLoaders:
loader_train = DisjointLoader(dataset_train, batch_size=32, shuffle=True)
loader_val = DisjointLoader(dataset_val, batch_size=32, shuffle=False)

# ---------------------------
# 5. Define the GIN Model in Keras (Disjoint Mode)
# ---------------------------
# In disjoint mode, we have three inputs:
# - Node features X: shape = (total_nodes, 1)
# - Adjacency matrix A: sparse, shape = (total_nodes, total_nodes)
# - Batch vector: shape = (total_nodes,)
input_X = Input(shape=(None, 1), name="X")              # Node features (variable number of nodes)
# IMPORTANT: For sparse input in disjoint mode, specify shape=(None,) not (None, None)
input_A = Input(shape=(None,), sparse=True, name="A")     # Sparse adjacency matrix (rank 2)
input_batch = Input(shape=(), dtype=tf.int32, name="batch")  # Batch vector

# Define a Sequential model for the GINConv layer's kernel/update function:
hidden_units = 32  # Number of hidden units in the MLP

x = GINConv(hidden_units, activation='relu', mlp_hidden=[hidden_units])([input_X, input_A])
x = GINConv(hidden_units, activation='relu', mlp_hidden=[hidden_units])([x, input_A])

# Global sum pooling using the batch vector:
def global_sum_pool(inputs):
    x, batch = inputs
    # Calculate the output shape dynamically:
    output_shape = tf.shape(x)[:-2] + [tf.shape(x)[-1]]  # (batch_size, hidden_units)
    # Perform segment sum and reshape to calculated shape
    return tf.reshape(tf.math.segment_sum(x, batch), output_shape)

# Apply global_sum_pool within a Lambda layer, explicitly specifying output_shape
x = Lambda(global_sum_pool, output_shape=(hidden_units,))([x, input_batch])

# Reshape or flatten the output before the final Dense layer
# x = Reshape((2,))(x)
x = Reshape((hidden_units,))(x) # Reshape to (batch_size, hidden_units)

# Final Dense layer for 2 outputs (ADHD_Outcome and Sex_F)
output = Dense(2, activation='sigmoid')(x)

gin_model = Model(inputs=[input_X, input_A, input_batch], outputs=output)
gin_model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
gin_model.summary()

# ---------------------------
# 6. Train the GIN Model
# ---------------------------
gin_model.fit(
    loader_train.load(),
    steps_per_epoch=loader_train.steps_per_epoch,
    validation_data=loader_val.load(),
    validation_steps=loader_val.steps_per_epoch,
    epochs=50,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

# ---------------------------
# 7. Validation: Compute Predictions and Weighted F1 Score
# ---------------------------
# Get a batch from the validation loader:
val_batch = next(loader_val.load())
X_val, y_val = val_batch  # X_val is a list: [X, A, batch]; y_val is shape (n_val, 2)
val_predictions = gin_model.predict(X_val)  # shape: (n_val, 2)

# Set thresholds (from threshold tuning)
best_thr_adhd = 0.38  # for ADHD_Outcome
best_thr_sex = 0.12   # for Sex_F

y_val_pred_adhd = (val_predictions[:, 0] > best_thr_adhd).astype(int)
y_val_pred_sex = (val_predictions[:, 1] > best_thr_sex).astype(int)

# Separate validation ground truth:
y_val_adhd = y_val[:, 0]
y_val_sex = y_val[:, 1]

def compute_weighted_f1(y_true_adhd, y_pred_adhd, y_true_sex, y_pred_sex):
    weights = np.ones_like(y_true_adhd, dtype=float)
    weights[(y_true_adhd == 1) & (y_true_sex == 1)] = 2.0
    f1_adhd = f1_score(y_true_adhd, y_pred_adhd, sample_weight=weights)
    f1_sex = f1_score(y_true_sex, y_pred_sex)
    final_f1 = (f1_adhd + f1_sex) / 2.0
    return final_f1, f1_adhd, f1_sex

final_f1, f1_adhd, f1_sex = compute_weighted_f1(y_val_adhd, y_val_pred_adhd, y_val_sex, y_val_pred_sex)
print(f"Final Weighted F1 Score: {final_f1:.4f}")
print(f"ADHD F1 Score (weighted): {f1_adhd:.4f}")
print(f"Sex_F F1 Score: {f1_sex:.4f}")

# ---------------------------
# 8. Create Submission CSV for Test Data
# ---------------------------
# # Assume test_mri is your DataFrame for test connectivity data,
# # and test_patient_ids is a NumPy array with participant IDs.
# # Extract connectivity data from test_mri (skip the first column: participant ID)
# test_matrix_data = test_mri.iloc[:, 1:].to_numpy(dtype=float)  # shape: (n_test, 19900)
# n_test = test_matrix_data.shape[0]
# X_graphs_test = []
# A_graphs_test = []
# for i in range(n_test):
#     Xg, A_dense = connectivity_vector_to_graph(test_matrix_data[i])
#     # Convert A_dense to a SciPy CSR sparse matrix
#     A_sparse = sp.csr_matrix(A_dense)
#     X_graphs_test.append(Xg)
#     A_graphs_test.append(A_sparse)

# # Create a list of Graph objects for the test set:
# from spektral.data import Graph
# test_graphs = [Graph(x=X_graphs_test[i], a=A_graphs_test[i]) for i in range(n_test)]
# # Create a DisjointLoader for the test dataset:
# loader_test = DisjointLoader(test_graphs, batch_size=32, shuffle=False)

# # Predict on test data:
# test_predictions = gin_model.predict(loader_test.load())  # shape: (n_test, 2)
# test_pred_adhd = (test_predictions[:, 0] > best_thr_adhd).astype(int)
# test_pred_sex = (test_predictions[:, 1] > best_thr_sex).astype(int)

# import pandas as pd
# submission_df = pd.DataFrame({
#     'participant_id': test_patient_ids.flatten(),
#     'ADHD_Outcome': test_pred_adhd,
#     'Sex_F': test_pred_sex
# })
# submission_path = '/content/drive/My Drive/widsdatathon2025/Work/Submission/submission.csv'
# submission_df.to_csv(submission_path, index=False)
# print(f"Submission file saved to: {submission_path}")


Model: "functional_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ X (InputLayer)            │ (None, None, 1)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ A (InputLayer)            │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gin_conv_29 (GINConv)     │ (None, None, 32)       │          1,249 │ X[0][0], A[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gin_conv_30 (GINConv)     │ (None, None, 32)       │          2,241 │ gin_conv_29[0][0],     │
│                           │                        │                │ A[0][0]                │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch (InputLayer)        │ (None)                 │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_6 (Lambda)         │ (None, 32)             │              0 │ gin_conv_30[0][0],     │
│                           │                        │                │ batch[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_9 (Reshape)       │ (None, 32)             │              0 │ lambda_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_61 (Dense)          │ (None, 2)              │             66 │ reshape_9[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,556 (13.89 KB)

 Trainable params: 3,428 (13.39 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node functional_47_1/gin_conv_29_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python

Pytorch

In [10]:
# Define GCN, GIN, GAT models

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATv2Conv, GINConv,  global_mean_pool,global_max_pool, global_add_pool, TopKPooling
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU
from torch_scatter import scatter_mean


MODELS = ['GCN','GIN', 'GAT']

def get_model_class(model_name):
    """Return the dataset class with the given name."""
    if model_name not in globals():
        raise NotImplementedError("Model not found: {}".format(model_name))
    return globals()[model_name]

def graph_readout(x,batch,  method):

    if method == 'mean':
        return global_mean_pool(x,batch)

    elif method == 'meanmax':
        x_mean = global_mean_pool(x,batch)
        x_max = global_max_pool(x,batch)
        return torch.cat((x_mean, x_max),1)

    elif method == 'sum':
        return global_add_pool(x,batch)

    else:
        raise ValueError('Undefined readout opertaion')

class Abstract_GNN(torch.nn.Module):
    """
    An Abstract class for all GNN models
    Subclasses should implement the following:
    - forward()
    - predict()
    """
    def __init__(self, num_nodes, f1, f2, readout):
        super(Abstract_GNN, self).__init__()
        self.readout = readout

    def _reset_parameters(self):
            for p in self.parameters():
                if p.dim() > 1:
                    nn.init.xavier_uniform_(p)
                else:
                    nn.init.uniform_(p)

    def forward(self,data):

        raise NotImplementedError


class GCN(Abstract_GNN):
    def __init__(self, num_nodes, f1, f2, readout, **kwargs):
        super().__init__(num_nodes, f1, f2, readout)
        self.readout = readout
        self.conv1 = GCNConv(num_nodes, f1)
        self.conv2 = GCNConv(f1, f2)

        last_dim = 2 if readout=='meanmax' else 1
        self.mlp = nn.Linear(f2*last_dim,2)
        self._reset_parameters()

    def forward(self, data):
        x, edge_index,batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = graph_readout(x, batch, self.readout)
        x = self.mlp(x)
        return x


class GIN(Abstract_GNN):
    def __init__(self, num_nodes, f1, f2, readout, extra_fc, **kwargs):
        super().__init__(num_nodes, f1, f2, readout)
        self.ll = extra_fc
        self.conv1 = GINConv(
            Sequential(Linear(num_nodes, f1), BatchNorm1d(f1), ReLU(),
                       Linear(f1, f1), ReLU()))

        self.conv2 = GINConv(
            Sequential(Linear(f1, f2), BatchNorm1d(f2), ReLU(),
                       Linear(f2, f2), ReLU()))

        last_dim = 2 if readout=='meanmax' else 1
        if self.ll:
            self.last = Sequential(Linear(f2*last_dim, f2*last_dim*2), ReLU(),
                                   Linear(f2*last_dim*2, 2))
        else:
            self.last = Linear(f2*last_dim, 2)

        self._reset_parameters()


    def forward(self, data):
        x, edge_index,batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = graph_readout(x, batch, self.readout)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.last(x)
        return x


class GAT(Abstract_GNN):
    def __init__(self, num_nodes, f1, f2, readout, num_heads, concat, **kwargs):
        super().__init__(num_nodes, f1, f2, readout)

        self.conv1 = GATv2Conv(num_nodes, f1, heads=num_heads, concat=concat)
        m = num_heads if concat else 1
        self.conv2 = GATv2Conv(f1*m, f2, heads=1)
        last_dim = 2 if readout=='meanmax' else 1
        self.mlp = nn.Linear(f2*last_dim,2)
        self._reset_parameters()


    def forward(self, data):
        x, edge_index,batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = graph_readout(x, batch, self.readout)
        x = self.mlp(x)
        return x

In [15]:
# Define train and test models

import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, Subset,  DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
import torch.nn.functional as F
import wandb
# from networks.utils import LabelSmoothingCrossEntropy
# from networks.static_models import *
# from networks.dynamic_models import *
from copy import deepcopy
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# from networks.utils import LabelSmoothingCrossEntropy

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, x, target, smoothing=0.1):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + smoothing * smooth_loss
        return loss.mean()



def train(model, train_loader, val_loader, config, device, a=None):

    model = model.to(device)

    '''model_hparams  = ["dropout", "dilation_exponential", "conv_channels", "layers", "adaptive", "readout",
    "jumping_knowledge",  "adj_norm",  "kernel_set", "bn", 'gcn', 'gcn_depth']

    model_params =  {key: config[key] for key in model_hparams}'''


    loss_function = LabelSmoothingCrossEntropy()
    optimizer = torch.optim.Adam(model.parameters(), config['lr'], weight_decay=1e-5)
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 400, eta_min = 1e-5)

    # start a typical PyTorch training
    best_metric = -1
    best_metric_epoch = -1
    best_val_loss = 1000
    best_model = None
    epochs = 1000

    print('-'*30)
    print ('Training ... ')
    early_stop = 30
    es_counter = 0

    for epoch in range(epochs):

        print("-" * 10)
        print(f"epoch {epoch + 1}/{epochs}")
        model.train()
        epoch_train_loss = 0

        for i , data in enumerate(tqdm(train_loader)):

            x = data.to(device)
            y = torch.tensor(data.y).type(torch.LongTensor).to(device)
            optimizer.zero_grad()

            out = model(x)
            step_loss = loss_function(out, y, smoothing=config['ls'])
            step_loss.backward(retain_graph=True)
            optimizer.step()
            epoch_train_loss += step_loss.item()

        epoch_train_loss = epoch_train_loss/(i+1)
        lr_scheduler.step()
        val_loss, val_acc = validate_model(model, val_loader, device)
        print(f"epoch {epoch + 1} train loss: {epoch_train_loss:.4f}")

        #if val_acc > best_metric and epoch>15:
        if val_loss < best_val_loss:
            best_metric = val_acc
            best_val_loss = val_loss
            best_metric_epoch = epoch + 1
            best_model = deepcopy(model)
            print("saved new best metric model")
            es_counter = 0
        else:
            es_counter += 1

        if es_counter > early_stop:
            print('No loss improvment.')
            break

        wandb_metric = {'train_loss':epoch_train_loss, 'val_loss':val_loss, 'val_acc': val_acc, 'best_val_loss': best_val_loss }
        wandb.log(wandb_metric)

        print(
            "current epoch: {} current val loss {:.4f} current accuracy: {:.4f}  best accuracy: {:.4f} at loss {:.4f} at epoch {}".format(
                epoch + 1, val_loss, val_acc, best_metric, best_val_loss, best_metric_epoch))

    print(f"train completed, best_val_loss: {best_val_loss:.4f} at epoch: {best_metric_epoch}")

    return best_model


def validate_model(model, val_loader, device):
    model.eval()
    val_loss = 0
    loss_func = nn.CrossEntropyLoss()

    labels = []
    preds = []
    for i, data in enumerate(val_loader):
            data = data.to(device)
            label = torch.tensor(data.y).type(torch.LongTensor).to(device)
            out = model(data)
            step_loss = loss_func(out, label)
            val_loss += step_loss.detach().item()
            preds.append(out.argmax(dim=1).detach().cpu().numpy())
            labels.append(label.cpu().numpy())
    preds = np.concatenate(preds).ravel()
    labels =  np.concatenate(labels).ravel()
    acc = balanced_accuracy_score(preds, labels)
    loss = val_loss/(i+1)

    return loss, acc

def test_model(model, test_loader, device):
    model.eval()
    labels = []
    preds = []
    for i, data in enumerate(test_loader):
            data = data.to(device)
            label = torch.tensor(data.y).type(torch.LongTensor).to(device)
            out = model(data)
            preds.append(out.argmax(dim=1).detach().cpu().numpy())
            labels.append(label.cpu().numpy())
    preds = np.concatenate(preds).ravel()
    labels =  np.concatenate(labels).ravel()
    acc = balanced_accuracy_score(labels, preds )
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    spec = tn / (tn+fp)
    sens = tp / (tp + fn)
    return acc, sens, spec

In [19]:
# from data import *
# from train import train, test_model
# from networks.static_models import *
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn import model_selection
import wandb
import warnings
import argparse
warnings.filterwarnings("ignore")

def prepare_ukbb_loaders(dataset,N, config):
    print('Preparing data ....')
    th = config['th']
    train_dataset = combined_metadata_imputed
    test_dataset = combined_test_metadata_imputed

    # stratify split for train and validation data
    train_labels = train_dataset.labels
    indices = list(range(len(train_labels)))
    train_idx, valid_idx = train_test_split(indices, test_size = 0.15, stratify = train_labels, random_state=0)
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], sampler = train_sampler,  num_workers=5, pin_memory=False)
    val_loader = DataLoader(train_dataset, batch_size=16, sampler = valid_sampler,  num_workers=5, pin_memory=False)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], num_workers=5, pin_memory=False)

    return train_loader, val_loader, test_loader


def run_1fold_ukbb(df_path, dataset,  model_name, config, device):
        dataset_name = dataset[0:4]
        N = dataset[5:]
        th = config['th']
        df = pd.read_csv(df_path)
        sample_dataset = get_dataset_class(dataset_name)(df[0:10], th, train = True)
        num_nodes = sample_dataset.nrois
        label = sample_dataset.LABEL


        model = get_model_class(model_name)(num_nodes, **config)
        train_loader, val_loader, test_loader  = prepare_ukbb_loaders(dataset_name,N,config)
        trained_model = train(model, train_loader, val_loader, config, device)
        accs, senss, specs = test_model(model, test_loader, device)
        print(" Test Accuracy = {:.2f}% \n Test Sens = {:.2f}% \n Test Spec = {:.2f}%".format( accs*100 , senss *100, specs*100))

        wandb.log({'Test_acc':accs, 'Test_sens':senss, 'Test_spec': specs})

def get_dataset_class(dataset_name):
    """Return the dataset class with the given name."""
    if dataset_name not in globals():
        raise NotImplementedError("Dataset not found: {}".format(dataset_name))
    return globals()[dataset_name]


def run_kfold(df_path, dataset,  model_name, config,  device):

    th = config['th']
    df = pd.read_csv(df_path)
    sample_dataset = get_dataset_class(dataset)(df[0:10], th, train = True)
    num_nodes = sample_dataset.nrois
    label = sample_dataset.LABEL
    skf = model_selection.StratifiedKFold(n_splits=5)
    skf.get_n_splits(df, df[label])

    accs = []
    senss = []
    specs = []
    k = 0

    for train_index, test_index in skf.split(df, df[label]):
        k+=1
        train_val_df = df.iloc[train_index]
        train_df, val_df = train_test_split(train_val_df, test_size = 0.15, stratify = train_val_df[label], random_state=0)
        test_df = df.iloc[test_index]

        train_loader = DataLoader(get_dataset_class(dataset)(train_df, th, train = True), batch_size=config['batch_size'],   num_workers=5, pin_memory=False)
        val_loader = DataLoader(get_dataset_class(dataset)(val_df, th, train = False), batch_size=config['batch_size'],   num_workers=5, pin_memory=False)
        test_loader = DataLoader(get_dataset_class(dataset)(test_df, th, train = False), batch_size=config['batch_size'], num_workers=5, pin_memory=False)
        model = get_model_class(model_name)(num_nodes, **config)

        trained_model = train(model, train_loader, val_loader, config, device)
        acc, sens, spec = test_model(model, test_loader, device)

        accs.append(acc)
        senss.append(sens)
        specs.append(spec)
        print("for fold {}, Acc = {:.3f}, Sens = {:.3f}, Spec  = {:.3f}".format(k,acc,sens,spec))
        print('-'*30)

    accs = np.array(accs)
    acc_mean = np.round(np.mean(accs),3)
    acc_std = np.round(np.std(accs),3)
    senss = np.array(senss)
    sens_mean = np.round(np.mean(senss),3)
    sens_std = np.round(np.std(senss),3)
    specs = np.array(specs)
    spec_mean = np.round(np.mean(specs),3)
    spec_std = np.round(np.std(specs),3)

    print (f'{model_name} on {dataset} dataset 5-fold results:')
    print(" Test Accuracy: mean = {:.3f} % ,std = {:.3f}".format(acc_mean*100, acc_std*100))
    print(" Test Sens: mean = {:.3f} % ,std = {:.3f}".format(sens_mean*100, sens_std*100))
    print(" Test Spec: mean = {:.3f} % ,std = {:.3f}".format(spec_mean*100, spec_std*100))

    wandb.log({'Test_acc':acc_mean, 'Test_sens':sens_mean, 'Test_spec': spec_mean, 'std': [acc_std, sens_std, spec_std]})

if __name__ == '__main__':

    # parser = argparse.ArgumentParser()
    # parser.add_argument('--device',type=str,default='cuda:1',help='device')
    # parser.add_argument('--dataset',type=str,default='Abide',help='Dataset name, valid options are ["Abide", "Mddrest", "UKBB_N"]')
    # parser.add_argument('--model',type=str,default='GCN', help='Static model name. valid options are ["GCN", "GIN", "GAT"]')

    # args = parser.parse_args()

    np.random.seed(0)
    # device = torch.device(args.device)
    device = torch.device('cuda:1')

    # dataset = args.dataset
    dataset = 'Abide'
    # model_name = args.model
    model_name = 'GIN'
    # df_paths = {'Mddrest': '../csvfiles/mddrest.csv'  , 'Abide':'../csvfiles/abide.csv',  'Ukbb_500': '../csvfiles/ukbb_500.csv',
    #  'Ukbb_1000': '../csvfiles/ukbb_1000.csv', 'Ukbb_2000': '../csvfiles/ukbb_2000.csv', 'Ukbb_5000': '../csvfiles/ukbb_5000.csv' }
    # # df_path = df_paths[args.dataset]
    # df_path = df_paths[dataset]

    #default hyperparameters, you can search for optimal conifguration using wandb sweep feature
    hparams_defaults = {"f1":128, "f2":64, 'layers': 3, 'readout': 'meanmax', 'batch_size': 32, 'lr': 3e-4,  'ls': 0.1, 'th': 0.5, 'extra_fc': True, 'num_heads': 1, 'concat': False}

    config = hparams_defaults
    # wandb.init(config = hparams_defaults, project= model_name+'-'+dataset)
    # config = wandb.config

    run_kfold(None, None, model_name, config, device)

    # print(f'Running a {model_name} on the {dataset} dataset.')
    # print('*'*50)
    # if dataset not in ['Abide', 'Mddrest']:
    #     run_1fold_ukbb(df_path, dataset,  model_name, config,  device)
    # else:
    #     run_kfold(df_path, dataset,  model_name, config,  device)

    print('-'*30)

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [34]:
train_matrix_data.shape

(1213, 19900)

In [ ]:
import numpy as np
import tensorflow as tf
import scipy.sparse as sp
from scipy.spatial.distance import squareform
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Lambda, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from spektral.data import Graph, Dataset, DisjointLoader
from spektral.layers import GINConv, GlobalSumPool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# ---------------------------
# 1. Define Helper Function: Convert a connectivity vector to a graph
# ---------------------------
def connectivity_vector_to_graph(vector, n=200):
    """
    Converts a connectivity vector (flattened upper-triangular part of a 200x200 matrix)
    into a graph representation:
      - X: Node feature matrix (here, constant ones, shape: [n, 1])
      - A: Adjacency matrix (shape: [n, n])
    """
    A = squareform(vector)
    np.fill_diagonal(A, 0)
    X = np.ones((n, 1))

    # Ensure A is in the correct format
    A = A.astype(np.float32)  # Ensure A is float32

    return X, A

# ---------------------------
# 2. Create a Custom Spektral Dataset
# ---------------------------
class FMRI_Dataset(Dataset):
    def __init__(self, connectivity_data, targets, **kwargs):
        """
        connectivity_data: NumPy array of shape (n_samples, 19900)
        targets: NumPy array of shape (n_samples, 2) with columns [ADHD_Outcome, Sex_F]
        """
        self.connectivity_data = connectivity_data
        self.targets = targets
        super().__init__(**kwargs)

    def read(self):
        output = []
        n_samples = self.connectivity_data.shape[0]
        for i in range(n_samples):
            X, A_dense = connectivity_vector_to_graph(self.connectivity_data[i])
            # Convert the dense adjacency matrix to a SciPy CSR sparse matrix:
            A_sparse = sp.csr_matrix(A_dense)
            y = self.targets[i]
            output.append(Graph(x=X, a=A_sparse, y=y))
        return output

# ---------------------------
# 3. Prepare the Training Data
# ---------------------------
# train_matrix_data: NumPy array of shape (1213, 19900)
# For example, if you previously defined it by merging metadata and fMRI connectivity data.
# Also, extract targets from train_sol:
y_adhd = train_sol['ADHD_Outcome'].values
y_sex  = train_sol['Sex_F'].values
y_target = np.column_stack((y_adhd, y_sex))

# Create the dataset
dataset = FMRI_Dataset(train_matrix_data, y_target)

# ---------------------------
# 4. Split the Dataset into Training and Validation Sets
# ---------------------------
n_samples = len(dataset)
train_idx, val_idx = train_test_split(np.arange(n_samples), test_size=0.2, random_state=42)
dataset_train = dataset[train_idx]
dataset_val = dataset[val_idx]

# Create DisjointLoaders
loader_train = DisjointLoader(dataset_train, batch_size=32, shuffle=True)
loader_val   = DisjointLoader(dataset_val, batch_size=32, shuffle=False)

# ---------------------------
# 5. Define the GIN Model in Keras (Disjoint Mode)
# ---------------------------
# In disjoint mode, the model will have three inputs:
# - X: node features (shape: (total_nodes, 1))
# - A: sparse adjacency matrix (shape: (total_nodes, total_nodes))
# - batch: a vector (shape: (total_nodes,)) indicating graph membership.
input_X = Input(shape=(None, 1), name="X")  # None = variable number of nodes per batch
input_A = Input(shape=(None,), sparse=True, name="A")
input_batch = Input(shape=(), dtype=tf.int32, name="batch")



# Two GINConv layers (using a simple Sequential block as kernel)
x = GINConv(32, activation='relu')([input_X, input_A])
x = GINConv(32, activation='relu')([x, input_A])
# Global pooling: GlobalSumPool aggregates node features based on the batch vector.
# (You can also use GlobalSumPool from Spektral or implement your own using tf.math.segment_sum)
x = GlobalSumPool()([x, input_batch])

# Reshape the output to match the target shape
x = Reshape((2,))(x)  # Reshape to (batch_size, 2)

# Final Dense layer for 2 outputs (ADHD_Outcome and Sex_F)
output = Dense(2, activation='sigmoid')(x)

gin_model = Model(inputs=[input_X, input_A, input_batch], outputs=output)
gin_model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
gin_model.summary()

# ---------------------------
# 6. Train the GIN Model
# ---------------------------
gin_model.fit(
    loader_train.load(),
    steps_per_epoch=loader_train.steps_per_epoch,
    validation_data=loader_val.load(),
    validation_steps=loader_val.steps_per_epoch,
    epochs=50,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

# ---------------------------
# 7. Validation: Compute Predictions and Weighted F1 Score
# ---------------------------
# Get validation predictions:
val_data = next(loader_val.load())
X_val, y_val = val_data  # X_val is a list: [X, A, batch], y_val is shape (n_val, 2)
val_predictions = gin_model.predict(X_val)  # shape: (n_val, 2)
# Set thresholds (from your threshold tuning):
best_thr_adhd = 0.38  # for ADHD_Outcome
best_thr_sex  = 0.12  # for Sex_F

y_val_pred_adhd = (val_predictions[:, 0] > best_thr_adhd).astype(int)
y_val_pred_sex  = (val_predictions[:, 1] > best_thr_sex).astype(int)

# Separate ground truth from y_val:
y_val_adhd = y_val[:, 0]
y_val_sex = y_val[:, 1]

def compute_weighted_f1(y_true_adhd, y_pred_adhd, y_true_sex, y_pred_sex):
    weights = np.ones_like(y_true_adhd, dtype=float)
    weights[(y_true_adhd == 1) & (y_true_sex == 1)] = 2.0
    f1_adhd = f1_score(y_true_adhd, y_pred_adhd, sample_weight=weights)
    f1_sex = f1_score(y_true_sex, y_pred_sex)
    final_f1 = (f1_adhd + f1_sex) / 2.0
    return final_f1, f1_adhd, f1_sex

final_f1, f1_adhd, f1_sex = compute_weighted_f1(y_val_adhd, y_val_pred_adhd, y_val_sex, y_val_pred_sex)
print(f"Final Weighted F1 Score: {final_f1:.4f}")
print(f"ADHD F1 Score (weighted): {f1_adhd:.4f}")
print(f"Sex_F F1 Score: {f1_sex:.4f}")

# ---------------------------
# 8. Create Submission CSV for Test Data
# ---------------------------
# Assume test_mri is a DataFrame for test connectivity data,
# and test_patient_ids is a NumPy array with test participant IDs.
# test_matrix_data = test_mri.iloc[:, 1:].to_numpy(dtype=float)  # shape: (n_test, 19900)
# n_test = test_matrix_data.shape[0]
# X_graphs_test = []
# A_graphs_test = []
# for i in range(n_test):
#     Xg, Ag = connectivity_vector_to_graph(test_matrix_data[i])
#     # Convert Ag to a SparseTensor:
#     Ag_sparse = tf.sparse.from_dense(Ag)
#     X_graphs_test.append(Xg)
#     A_graphs_test.append(Ag_sparse)

# # Create a list of Graph objects for the test set:
# from spektral.data import Graph
# test_graphs = [Graph(x=X_graphs_test[i], a=A_graphs_test[i]) for i in range(n_test)]
# # Create a DisjointLoader for the test dataset:
# loader_test = DisjointLoader(test_graphs, batch_size=32, shuffle=False)

# # Predict on the test data:
# test_predictions = gin_model.predict(loader_test.load())  # shape: (n_test, 2)
# test_pred_adhd = (test_predictions[:, 0] > best_thr_adhd).astype(int)
# test_pred_sex = (test_predictions[:, 1] > best_thr_sex).astype(int)

# # Create a submission DataFrame:
# import pandas as pd
# submission_df = pd.DataFrame({
#     'participant_id': test_patient_ids.flatten(),  # ensure a 1D array of IDs
#     'ADHD_Outcome': test_pred_adhd,
#     'Sex_F': test_pred_sex
# })
# submission_path = '/content/drive/My Drive/widsdatathon2025/Work/Submission/submission.csv'
# submission_df.to_csv(submission_path, index=False)
# print(f"Submission file saved to: {submission_path}")


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ X (InputLayer)            │ (None, None, 1)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ A (InputLayer)            │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gin_conv_17 (GINConv)     │ (None, None, 32)       │             65 │ X[0][0], A[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gin_conv_18 (GINConv)     │ (None, None, 32)       │          1,057 │ gin_conv_17[0][0],     │
│                           │                        │                │ A[0][0]                │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch (InputLayer)        │ (None)                 │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_sum_pool_6         │ (None, None, 32)       │              0 │ gin_conv_18[0][0],     │
│ (GlobalSumPool)           │                        │                │ batch[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_4 (Reshape)       │ (None, 2)              │              0 │ global_sum_pool_6[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_27 (Dense)          │ (None, 2)              │              6 │ reshape_4[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,128 (4.41 KB)

 Trainable params: 1,128 (4.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'FMRI_Dataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


InvalidArgumentError: Graph execution error:

Detected at node functional_19_1/gin_conv_17_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python

In [ ]:
import numpy as np
import tensorflow as tf
from scipy.spatial.distance import squareform
import scipy.sparse as sp
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Lambda, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from spektral.data import Graph, Dataset, DisjointLoader
from spektral.layers import GINConv, GlobalSumPool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# ---------------------------
# 1. Helper: Convert a connectivity vector to a graph
# ---------------------------
def connectivity_vector_to_graph(vector, n=200):
    """
    Converts a connectivity vector (flattened upper-triangular part of a 200x200 matrix)
    into a graph representation:
      - X: Node features (all ones, shape: [n, 1])
      - A: Adjacency matrix (dense, shape: [n, n])
    """
    A = squareform(vector)
    np.fill_diagonal(A, 0)
    X = np.ones((n, 1))
    A = A.astype(np.float32)
    return X, A

# ---------------------------
# 2. Custom Spektral Dataset
# ---------------------------
class FMRI_Dataset(Dataset):
    def __init__(self, connectivity_data, targets, **kwargs):
        """
        connectivity_data: NumPy array of shape (n_samples, 19900)
        targets: NumPy array of shape (n_samples, 2) with columns [ADHD_Outcome, Sex_F]
        """
        self.connectivity_data = connectivity_data
        self.targets = targets
        super().__init__(**kwargs)

    def read(self):
        output = []
        n_samples = self.connectivity_data.shape[0]
        for i in range(n_samples):
            X, A_dense = connectivity_vector_to_graph(self.connectivity_data[i])
            # Convert A_dense to a SciPy CSR sparse matrix:
            A_sparse = sp.csr_matrix(A_dense)
            y = self.targets[i]
            output.append(Graph(x=X, a=A_sparse, y=y))
        return output

# ---------------------------
# 3. Prepare Training Data
# ---------------------------
# Assume train_mri has shape (1213, 19901): first column is ID, next 19900 columns are connectivity.
train_matrix_data = train_mri.iloc[:, 1:].to_numpy(dtype=float)  # shape: (1213, 19900)

# Extract targets from train_sol:
y_adhd = train_sol['ADHD_Outcome'].values  # shape: (1213,)
y_sex  = train_sol['Sex_F'].values         # shape: (1213,)
y_target = np.column_stack((y_adhd, y_sex))  # shape: (1213, 2)

# Create the dataset:
dataset = FMRI_Dataset(train_matrix_data, y_target)

# ---------------------------
# 4. Split Dataset into Training and Validation Sets
# ---------------------------
n_samples = len(dataset)
train_idx, val_idx = train_test_split(np.arange(n_samples), test_size=0.2, random_state=42)
dataset_train = dataset[train_idx]
dataset_val = dataset[val_idx]

# Create DisjointLoaders:
loader_train = DisjointLoader(dataset_train, batch_size=32, shuffle=True)
loader_val   = DisjointLoader(dataset_val, batch_size=32, shuffle=False)

# ---------------------------
# 5. Define the GIN Model (Disjoint Mode) in Keras
# ---------------------------
# Disjoint mode inputs:
# - input_X: node features, shape = (total_nodes, 1)
# - input_A: sparse adjacency matrix, shape = (total_nodes, total_nodes)
# - input_batch: batch vector, shape = (total_nodes,)
input_X = Input(shape=(None, 1), name="X")
input_A = Input(shape=(None, None), sparse=True, name="A")  # Corrected shape here!
input_batch = Input(shape=(), dtype=tf.int32, name="batch")

# Two GINConv layers:
x = GINConv(Dense(32, activation='relu'))([input_X, input_A])
x = GINConv(Dense(32, activation='relu'))([x, input_A])
# Global pooling: sum pooling using GlobalSumPool with the batch vector
x = GlobalSumPool()([x, input_batch])
x = Reshape((-1,))(x)  # Flatten the pooled representation

# Final Dense layer for 2 outputs (ADHD_Outcome and Sex_F)
output = Dense(2, activation='sigmoid')(x)

gin_model = Model(inputs=[input_X, input_A, input_batch], outputs=output)
gin_model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
gin_model.summary()

# ---------------------------
# 6. Train the GIN Model
# ---------------------------
gin_model.fit(
    loader_train.load(),
    steps_per_epoch=loader_train.steps_per_epoch,
    validation_data=loader_val.load(),
    validation_steps=loader_val.steps_per_epoch,
    epochs=50,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

# ---------------------------
# 7. Validation: Compute Predictions and Weighted F1 Score
# ---------------------------
# Get a batch from the validation loader:
val_batch = next(loader_val.load())
X_val, y_val = val_batch  # X_val is a list: [X, A, batch]; y_val shape: (n_val, 2)
val_predictions = gin_model.predict(X_val)  # shape: (n_val, 2)

# Set thresholds (from tuning)
best_thr_adhd = 0.38
best_thr_sex  = 0.12

y_val_pred_adhd = (val_predictions[:, 0] > best_thr_adhd).astype(int)
y_val_pred_sex  = (val_predictions[:, 1] > best_thr_sex).astype(int)

# Separate ground truth:
y_val_adhd = y_val[:, 0]
y_val_sex = y_val[:, 1]

def compute_weighted_f1(y_true_adhd, y_pred_adhd, y_true_sex, y_pred_sex):
    weights = np.ones_like(y_true_adhd, dtype=float)
    weights[(y_true_adhd == 1) & (y_true_sex == 1)] = 2.0
    f1_adhd = f1_score(y_true_adhd, y_pred_adhd, sample_weight=weights)
    f1_sex = f1_score(y_true_sex, y_pred_sex)
    final_f1 = (f1_adhd + f1_sex) / 2.0
    return final_f1, f1_adhd, f1_sex

final_f1, f1_adhd, f1_sex = compute_weighted_f1(y_val_adhd, y_val_pred_adhd, y_val_sex, y_val_pred_sex)
print(f"Final Weighted F1 Score: {final_f1:.4f}")
print(f"ADHD F1 Score (weighted): {f1_adhd:.4f}")
print(f"Sex_F F1 Score: {f1_sex:.4f}")

# ---------------------------
# 8. Create Submission CSV for Test Data
# ---------------------------
# Assume test_mri is your DataFrame for test connectivity data,
# and test_patient_ids is a NumPy array with test participant IDs.
test_matrix_data = test_mri.iloc[:, 1:].to_numpy(dtype=float)  # shape: (n_test, 19900)
n_test = test_matrix_data.shape[0]
X_graphs_test = []
A_graphs_test = []
for i in range(n_test):
    Xg, A_dense = connectivity_vector_to_graph(test_matrix_data[i])
    # Convert A_dense to a SciPy sparse matrix
    A_sparse = sp.csr_matrix(A_dense)
    X_graphs_test.append(Xg)
    A_graphs_test.append(A_sparse)

# Create a list of Graph objects for the test set:
from spektral.data import Graph
test_graphs = [Graph(x=X_graphs_test[i], a=A_graphs_test[i]) for i in range(n_test)]
# Create a DisjointLoader for the test dataset:
loader_test = DisjointLoader(test_graphs, batch_size=32, shuffle=False)

# Predict on test data:
test_predictions = gin_model.predict(loader_test.load())  # shape: (n_test, 2)
test_pred_adhd = (test_predictions[:, 0] > best_thr_adhd).astype(int)
test_pred_sex  = (test_predictions[:, 1] > best_thr_sex).astype(int)

import pandas as pd
submission_df = pd.DataFrame({
    'participant_id': test_patient_ids.flatten(),
    'ADHD_Outcome': test_pred_adhd,
    'Sex_F': test_pred_sex
})
submission_path = '/content/drive/My Drive/widsdatathon2025/Work/Submission/submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"Submission file saved to: {submission_path}")


AssertionError: Exception encountered when calling GINConv.call().

[1mCould not automatically infer the output shape / dtype of 'gin_conv_15' (of type GINConv). Either the `GINConv.call()` method is incorrect, or you need to implement the `GINConv.compute_output_spec() / compute_output_shape()` method. Error encountered:

A must have rank 2[0m

Arguments received by GINConv.call():
  • args=(['<KerasTensor shape=(None, None, 1), dtype=float32, sparse=False, name=X>', '<KerasTensor shape=(None, None, None), dtype=float32, sparse=True, name=A>'],)
  • kwargs=<class 'inspect._empty'>